In [41]:
from typing import List, DefaultDict

import numpy as np
from konlpy.tag._komoran import Komoran

# 문서 유사도
원리: 두 문장 간 유사한 단어들이 얼마나 겹치는지 정도를 측정

이 프로젝트는 QnA 챗봇을 만듦을 목표로 한다. 챗봇은 몇 가지 질문에 대한 답변을 가지고 있다.
유저가 챗봇에게 질문을 하면 챗봇은 이 질문을 임베딩 변환해서 자기가 답변을 아는 질문 중 가장 유사한 것을
찾아낸다. 찾아낸 가장 유사한 질문에 대한 답이 원래 유저가 요구했던 답변이라고 판단하는 것이다.

## n-gram 유사도
- n-gram의 예시(n=2, 문장 단위 문서 '아버지가 신문을 들고 방에서 나오며 나를 찾았다.'를 예로)
1. 단어(명사) 추출: `['아버지', '신문', 방', '나']`
2. TDM(Term-Document Matrix) 생성:

| 아버지 | 신문  | 방 | 나   | 토큰        |
|------|-----|----|-----|-----------|
|   1  | 1   |  0  | 0   | (아버지, 신문) |
|   0  | 1   |  1  | 0   | (신문, 방)   |
|   0  | 0   |  1  | 1   | (방, 나)    |

3. 토큰 정리: `[['아버지', '신문'], ['신문', '방'], ['방', '나']]`

- n-gram 기반 유사도 계산
> `similarity(doc1, doc2) = tf(doc1, doc2) / tokens(doc1)`
> `tf(doc1, doc2)`는 term frequency로서 doc1과 doc2에서 동시에 나타나는 토큰의 빈도,
> `tokens(doc1)`는 doc1 내 모든 토큰의 수

In [11]:
def word_ngram(bag_of_words: List, num_gram: int) -> List:
    """n-gram 토큰 나열

    :param bag_of_words: 문서 내 구별되는 단어의 나열.
    :param num_gram: n-gram에서 n의 크기.
    :return: 토큰 리스트.
    """
    ngrams = [bag_of_words[i:i + num_gram] for i in range(len(bag_of_words))]
    return ngrams


def n_gram_similarity(doc1: List, doc2: List) -> float:
    """n-gram 기반 유사도 계산

    :param doc1: `word_ngram`의 리턴. 이 문서가 doc2와 얼마나 유사한지 계산한다.
    :param doc2: `word_ngram`의 리턴
    :return: 유사도
    """
    num_common_tokens = 0
    for token in doc1:
        if token in doc2:
            num_common_tokens += 1
    num_tokens_in_doc1 = len(doc1)

    return num_common_tokens / num_tokens_in_doc1

In [19]:
sentence1 = "리바운드를 제압하는 자가 농구를 제압한다."
sentence2 = "화력으로 제압하는 부대가 상대를 제압한다."
sentence3 = "리바운드를 잘하는 자가 농구에 필요하다."

In [6]:
komoran = Komoran()

In [76]:
nouns1 = komoran.nouns(sentence1)
nouns2 = komoran.nouns(sentence2)
nouns3 = komoran.nouns(sentence3)

In [21]:
nouns1

['리바운드', '제압', '자', '농구', '제압']

In [34]:
num_gram = 2
tokens1 = word_ngram(nouns1, num_gram)
tokens2 = word_ngram(nouns2, num_gram)
tokens3 = word_ngram(nouns3, num_gram)

In [35]:
print(tokens1)
print(tokens2)
print(tokens3)

[['리바운드', '제압'], ['제압', '자'], ['자', '농구'], ['농구', '제압'], ['제압']]
[['화력', '제압'], ['제압', '부대'], ['부대', '상대'], ['상대', '제압'], ['제압']]
[['리바운드', '자'], ['자', '농구'], ['농구', '필요'], ['필요']]


In [36]:
gram_sim12 = n_gram_similarity(tokens1, tokens2)
gram_sim23 = n_gram_similarity(tokens2, tokens3)
gram_sim31 = n_gram_similarity(tokens3, tokens1)

In [37]:
print(f"'{sentence1}' '{sentence2}'의 유사도: {gram_sim12}")
print(f"'{sentence2}' '{sentence3}'의 유사도: {gram_sim23}")
print(f"'{sentence1}' '{sentence3}'의 유사도: {gram_sim31}")

'리바운드를 제압하는 자가 농구를 제압한다.' '화력으로 제압하는 부대가 상대를 제압한다.'의 유사도: 0.2
'화력으로 제압하는 부대가 상대를 제압한다.' '리바운드를 잘하는 자가 농구에 필요하다.'의 유사도: 0.0
'리바운드를 제압하는 자가 농구를 제압한다.' '리바운드를 잘하는 자가 농구에 필요하다.'의 유사도: 0.2


## 코사인 유사도
두 임베딩 벡터 간 코사인 값으로 유사도를 측정.
  - 두 벡터가 정반대: cos = -1
  - 두 벡터가 직교: cos = 0
  - 두 벡터가 같은 방향: cos = 1

`similarity = cos = dot(v1, v2) / norm(v1) * norm(v2)`이다.

In [111]:
def cos_similarity(v1, v2):
    """코사인 유사도 계산

    :param v1: 임베딩 벡터 1
    :param v2: 임베딩 벡터 2
    :return: v1, v2 간 유사도
    """
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


def term_document_matrix(sentence_tokens, word_set) -> dict:
    """문장 내 토큰이 문서에서 몇 번 나왔는지 기록.

    :param sentence_tokens: 특정 문장 내 토큰들.
    :param word_set: sentence_tokens의 문장을 포함한 문서 내 구별되는 단어(명사) 집합.
    :return: `word_set` 내에서 토큰별 빈도를 기록.
    """
    tdm = {word: 0 for word in word_set}

    for word in word_set:
        if word in sentence_tokens:
            tdm[word] += 1

    return tdm


def vectorize(tdm: DefaultDict):
    """토큰 출현 빈도로 벡터 생성.

    :param tdm: `term_document_matrix`의 리턴값.
    :return: 문서의 단어 집합에서 토큰이 몇 번 나왔는지를 나열한 리스트.
    """
    vector = []
    for freq in tdm.values():
        vector.append(freq)

    return vector

In [93]:
word_set = set(nouns1) | set(nouns2) | set(nouns3)

In [100]:
# 위에서 'token'을 n-gram의 단위를 칭하는데 썼기에 대신, bag-of-words로 문장 내 단어 집합 칭한다.
bow1 = komoran.nouns(sentence1)
bow2 = komoran.nouns(sentence2)
bow3 = komoran.nouns(sentence3)

In [112]:
token_freq1 = term_document_matrix(bow1, word_set)
token_freq2 = term_document_matrix(bow2, word_set)
token_freq3 = term_document_matrix(bow3, word_set)

In [114]:
vec1 = vectorize(token_freq1)
vec2 = vectorize(token_freq2)
vec3= vectorize(token_freq3)

In [116]:
vec1

[0, 1, 0, 0, 1, 0, 1, 1]

In [119]:
cos_sim12 = cos_similarity(vec1, vec2)
cos_sim23 = cos_similarity(vec2, vec3)
cos_sim31 = cos_similarity(vec3, vec1)

In [121]:
print(f"'{sentence1}' '{sentence2}'의 유사도: {cos_sim12}")
print(f"'{sentence2}' '{sentence3}'의 유사도: {cos_sim23}")
print(f"'{sentence1}' '{sentence3}'의 유사도: {cos_sim31}")

'리바운드를 제압하는 자가 농구를 제압한다.' '화력으로 제압하는 부대가 상대를 제압한다.'의 유사도: 0.25
'화력으로 제압하는 부대가 상대를 제압한다.' '리바운드를 잘하는 자가 농구에 필요하다.'의 유사도: 0.0
'리바운드를 제압하는 자가 농구를 제압한다.' '리바운드를 잘하는 자가 농구에 필요하다.'의 유사도: 0.75
